In [1]:
import dxpy
import dxdata
import subprocess
import ast
import numpy as np
import pandas as pd
import glob 
import os

In [2]:
#####################################################################
#####################################################################
#Returning dosage information                               #########
#####################################################################
#####################################################################

In [3]:
pip install pybgen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 7.6 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install zstandard

Note: you may need to restart the kernel to use updated packages.


In [5]:
# #split string input of multiple SNPs
# def split_input(string_input):
#     parts = string_input.split(':')
#     chrom = parts[0]
#     pos = int(parts[1])
#     ref = parts[2]
#     alt = parts[3]
#     rs_num = parts[4]
#     return chrom, pos, ref, alt, rs_num


In [5]:
#split string input of multiple SNPs
def split_input(string_input):
    if ':' in string_input:
        parts = string_input.split(':')
    elif '-' in string_input:
        parts = string_input.split('-')
    chrom = parts[0]
    pos = int(parts[1])
    ref = parts[2]
    alt = parts[3]
    if len(parts) > 4:
        rsnum = parts[4]
        return chrom,pos,ref,alt,rsnum
    else:
        return chrom, pos, ref, alt


In [6]:
def split_input_from_CSV(dataframe_CSV): 
    return None
    

In [7]:
def getImputedPath(chrom):
    imputed_path = '/mnt/project/Bulk/Imputation/Imputation from genotype (TOPmed)/ukb21007_c' + chrom[3:] + '_b0_v1.bgen'
    return imputed_path

In [8]:
def getWESPath(chrom):
    wes_path = '/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c' +chrom[3:]+ '_b0_v1.bgen'
    return wes_path

In [9]:
def getSampleDF(chrom):
    sample_path = '/mnt/project/Bulk/Imputation/Imputation from genotype (TOPmed)/ukb21007_c' + chrom[3:] + '_b0_v1.sample'
    sample_df = pd.read_csv(sample_path, sep=' ')
    sample_df = sample_df.drop(index=0)
    sample_df = sample_df.drop(['missing', 'sex'], axis=1)
    sample_df = sample_df.reset_index(drop=True)
    sample_df = sample_df.rename(columns={"ID_1": "FID", "ID_2": "IID"}, errors="raise")
    return sample_df

In [10]:
def getWESSampleDF(chrom):
    sample_path = '/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, BGEN format - final release/ukb23159_c' + chrom[3:] + '_b0_v1.sample'
    sample_df = pd.read_csv(sample_path, sep=' ')
    sample_df = sample_df.drop(index=0)
    sample_df = sample_df.drop(['missing', 'sex'], axis=1)
    sample_df = sample_df.reset_index(drop=True)
    sample_df = sample_df.rename(columns={"ID_1": "FID", "ID_2": "IID"}, errors="raise")
    return sample_df

In [11]:
# synpo2L['rounded'] = synpo2L['rs3740293'].round()

# # Calculate the frequencies of rounded numbers in column 'rounded_c'
# frequency_counts = synpo2L['rounded'].value_counts()

# # Print the frequencies
# print(frequency_counts)

In [13]:

# rodeoSNPs = pd.read_csv('/mnt/project/TTN/dosage_extraction/wrangler_SNPs.txt', delimiter='\t')
# # rodeoSNPs = rodeoSNPs.dropna(subset=['Chr'])

# def combine_columns(row):
#     chr_str = "chr" + str(int(row['Chr']))
#     pos_str = str(int(row['pos']))
#     return f"{chr_str}:{pos_str}:{row['ref_allele']}:{row['alt_alleles']}:{row['SNP']}"

# # Apply the function to each row and create a new column 'combined'
# interestingSNPs = rodeoSNPs.apply(combine_columns, axis=1).tolist()

# # Display the list of combined strings
# print(interestingSNPs)

In [14]:
BAG3vars = pd.read_csv('/mnt/project/TTN/dosages/BAG3_WES_CADD20_toExtract.csv', delimiter=',')
MYL3vars = pd.read_csv('/mnt/project/TTN/dosages/MYL3_WES_toExtract.csv', delimiter = ',')

In [15]:
BAG3vars.head(100)

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,phastCons100way_vertebrate,phastCons100way_vertebrate_rankscore,phastCons20way_mammalian,phastCons20way_mammalian_rankscore,SiPhy_29way_logOdds,SiPhy_29way_logOdds_rankscore,Interpro_domain,GTEx_V6_gene,GTEx_V6_tissue,Otherinfo1
0,10,119676935,119676935,G,T,exonic,BAG3,.,stopgain,BAG3:NM_004281:exon4:c.G1381T:p.E461X,...,1.000,0.715,1.000,0.888,19.960,0.972,BAG domain,.,.,NaN
1,10,119676971,119676971,C,T,exonic,BAG3,.,stopgain,BAG3:NM_004281:exon4:c.C1417T:p.R473X,...,0.998,0.411,1.000,0.888,16.208,0.819,BAG domain,.,.,NaN
2,10,119676479,119676479,C,T,exonic,BAG3,.,stopgain,BAG3:NM_004281:exon4:c.C925T:p.R309X,...,0.996,0.391,1.000,0.888,16.271,0.824,.,.,.,NaN
3,10,119672467,119672467,C,G,exonic,BAG3,.,stopgain,BAG3:NM_004281:exon3:c.C720G:p.Y240X,...,1.000,0.715,1.000,0.888,16.748,0.853,.,.,.,NaN
4,10,119670106,119670106,C,T,exonic,BAG3,.,stopgain,BAG3:NM_004281:exon2:c.C436T:p.Q146X,...,0.989,0.365,0.934,0.407,18.547,0.910,.,.,.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10,119669969,119669969,A,G,exonic,BAG3,.,nonsynonymous SNV,BAG3:NM_004281:exon2:c.A299G:p.H100R,...,1.000,0.715,0.940,0.414,15.446,0.748,.,.,.,NaN
96,10,119672462,119672462,G,A,exonic,BAG3,.,nonsynonymous SNV,BAG3:NM_004281:exon3:c.G715A:p.E239K,...,1.000,0.715,1.000,0.888,16.690,0.851,.,.,.,NaN
97,10,119672628,119672628,G,A,exonic,BAG3,.,nonsynonymous SNV,BAG3:NM_004281:exon3:c.G881A:p.R294H,...,1.000,0.715,0.738,0.318,10.046,0.412,.,.,.,NaN
98,10,119676569,119676569,A,T,exonic,BAG3,.,nonsynonymous SNV,BAG3:NM_004281:exon4:c.A1015T:p.I339F,...,0.999,0.424,0.990,0.544,16.302,0.826,.,.,.,NaN


In [16]:
cols = ['Chr', 'Start', 'Ref', 'Alt']

In [17]:
MYL3vars['variant_id'] = MYL3vars[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
BAG3vars['variant_id'] = BAG3vars[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)

In [18]:
interestingSNPs = []
for var_id in MYL3vars['variant_id']:
    modified_var_id = "chr" + var_id
    interestingSNPs.append(modified_var_id)

# Display the list of combined strings
print(interestingSNPs)

['chr3-46859601-G-A', 'chr3-46859543-C-T', 'chr3-46860741-C-T', 'chr3-46859535-C-T', 'chr3-46859544-G-A', 'chr3-46860694-C-A', 'chr3-46860742-G-A', 'chr3-46860795-C-G', 'chr3-46860795-C-T', 'chr3-46858411-C-T', 'chr3-46858441-C-T', 'chr3-46859496-G-A', 'chr3-46860703-G-A', 'chr3-46860781-C-T', 'chr3-46860796-G-A', 'chr3-46860799-C-A', 'chr3-46859490-C-T', 'chr3-46859525-C-A', 'chr3-46859556-C-T', 'chr3-46860702-C-G', 'chr3-46860738-G-A', 'chr3-46860739-C-T', 'chr3-46858413-T-A', 'chr3-46860732-C-T', 'chr3-46860758-C-A', 'chr3-46860813-G-T', 'chr3-46860751-C-A', 'chr3-46859609-G-A', 'chr3-46859480-G-A', 'chr3-46858413-T-C', 'chr3-46860822-A-G', 'chr3-46858387-C-T', 'chr3-46859614-G-C', 'chr3-46859546-A-T', 'chr3-46860727-T-A', 'chr3-46858251-G-A', 'chr3-46860763-C-T', 'chr3-46859500-C-A', 'chr3-46859540-A-G', 'chr3-46860724-G-A', 'chr3-46859495-C-T', 'chr3-46860804-A-T', 'chr3-46859490-C-A', 'chr3-46859556-C-A', 'chr3-46860805-G-C', 'chr3-46863269-T-A', 'chr3-46863272-G-A', 'chr3-468607

In [19]:
# from pybgen import PyBGEN
# def getVariantDosagesfromSnp(interestingSNPs):
#     sample_df = None
#     for snpOfChoice in interestingSNPs:
#         chrom, pos, ref, alt, rsNum = split_input(snpOfChoice)
#         imputed_path = getImputedPath(chrom)
#         with PyBGEN(imputed_path) as bgen:
#             try:
#                 variant = bgen.get_specific_variant(chrom, pos, ref, alt)
#             except ValueError:
#                 print(f"Variant {snpOfChoice} not found. Skipping.")
#                 continue 
#         if sample_df is None:
#             sample_df = getSampleDF(chrom)
#         chromosome = chrom 
#         position = str(pos)
#         temp_df = pd.DataFrame(variant[0][1], columns= [rsNum])
#         dosage_df = pd.concat([sample_df, temp_df], axis=1)
#         sample_df = dosage_df
#     return dosage_df

In [27]:
#using wes data
from pybgen import PyBGEN
def getVariantDosagesfromSnp(interestingSNPs):
    sample_df = None
    for snpOfChoice in interestingSNPs:
        chrom, pos, ref, alt, snp = split_input(snpOfChoice)
        imputed_path = getImputedPath(chrom)
        wes_path = getWESPath(chrom)
        chromNum = chrom[3:]
        if chromNum == 'X':
            chromNum = 23
        with PyBGEN(imputed_path) as bgen:
            try:
                variant = bgen.get_specific_variant(chrom, pos, ref, alt) #Use chromNum if WES, chrom if imputed
            except ValueError:
                print(f"Variant {snp} not found. Skipping.")
                continue 
        if sample_df is None:
            sample_df = getWESSampleDF(chrom)
        chromosome = chrom 
        position = str(pos)
        temp_df = pd.DataFrame(variant[0][1], columns= [snp])
        dosage_df = pd.concat([sample_df, temp_df], axis=1)
        sample_df = dosage_df
    return dosage_df

In [138]:
from pybgen import PyBGEN
#Converts a list of chr, pos, rsnum to a list of variants with alleles found in the bgen file 

def getVariants(snplocs):
    newsnps = []
    foundsnps = []
    for snp in snplocs:
        chrom, pos, rsnum = snp.split(':')
        imputed_path = getImputedPath(chrom)
        wes_path = getWESPath(chrom)
        chromNum = chrom[3:]
        if chromNum == 'X':
            chromNum = 23
        with PyBGEN(imputed_path) as bgen: 
            val = bgen.iter_variants_in_region(chrom, pos, pos)
            for v in val:
                print(np.nansum(v[1]))
                ref = (str(v[0]).split('_')[1].split('/')[0])
                alt = (str(v[0]).split('_')[1].split('/')[1]).strip('>')
                combined = ':'.join([chrom, pos, ref, alt, rsnum])
                found = ':'.join([chrom, pos, rsnum])
                #print(combined)
                newsnps.append(combined)
                foundsnps.append(found)
    missedsnps = list(set(snplocs) - set(foundsnps))
    for bad in missedsnps:
        print(bad + ' not found')
        
    return newsnps
            

In [141]:
test = ['chr17:39696844:rs2517951',
'chr1:109278685:rs583104',
'chr10:119654724:rs196336',
'chr1:109275684:rs629301']
#test = [          'chr5:-100:haha']
print(getVariants(test))

648048.4352941174
741129.9333333331
578266.4156862743
755580.3411764707
['chr17:39696844:C:T:rs2517951', 'chr1:109278685:G:T:rs583104', 'chr10:119654724:T:C:rs196336', 'chr1:109275684:G:T:rs629301']


In [30]:
#HMGCR 

HMGCR_snps = ['chr5:75360714:T:C:rs12916', 
             'chr5:75345735:G:A:rs10515198',
             'chr5:75401225:T:G:rs12173076',
             'chr5:75324552:T:C:rs3857388', 
             'chr5:75244572:A:G:rs7711235']

multipleSNPdosages = getVariantDosagesfromSnp(HMGCR_snps)
multipleSNPdosages

(<Variant . chrchr5:75360714_T/C>, array([1., 0., 1., ..., 1., 0., 1.]))


/tmp/ipykernel_98/1515039186.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv(sample_path, sep=' ')


(<Variant . chrchr5:75345735_G/A>, array([1., 0., 0., ..., 0., 0., 0.]))
(<Variant . chrchr5:75401225_T/G>, array([0., 0., 0., ..., 0., 0., 1.]))
(<Variant . chrchr5:75324552_T/C>, array([0., 0., 1., ..., 0., 0., 0.]))
(<Variant . chrchr5:75324552_T/TC>, array([0., 0., 0., ..., 0., 0., 0.]))
(<Variant . chrchr5:75244572_A/G>, array([1., 0., 1., ..., 0., 1., 0.]))


In [30]:
#ADRB1

ADRB1_snps = ['chr10:114044277:A:G:rs1801252', 
             'chr10:114045297:G:C:rs1801253']

multipleSNPdosages = getVariantDosagesfromSnp(ADRB1_snps)
multipleSNPdosages

Variant rs1801252 not found. Skipping.
Variant rs1801253 not found. Skipping.


UnboundLocalError: local variable 'dosage_df' referenced before assignment

In [ ]:
#ADRB2
ADRB2_snps = ['chr5:148826910:G:C:rs1042714', 
             'chr5:148826877:G:A:rs1042713']

multipleSNPdosages = getVariantDosagesfromSnp(ADRB2_snps)
multipleSNPdosages

In [ ]:
#ACE
ACE_snps = ['chr17:63488670:G:A:rs4343']

multipleSNPdosages = getVariantDosagesfromSnp(ACE_snps)
multipleSNPdosages

In [ ]:
#ACE2
ACE2_snps = ['chrX:15592225:C:T:rs2285666']

multipleSNPdosages = getVariantDosagesfromSnp(ACE2_snps)
multipleSNPdosages

In [18]:
#ACE1
ACE1_snps = ['chr17:63422371:C:T:rs116112765',
            'chr17:63480412:A:G:rs3730025',
            'chr17:63493061:A:G:rs4353',
            'chr17:63497131:G:A:rs4364',
            'chr17:63471115:G:A:rs4968782']

multipleSNPdosages = getVariantDosagesfromSnp(ACE1_snps)
multipleSNPdosages

,FID,IID,rs116112765,rs3730025,rs4353,rs4364,rs4968782
0,3253844,3253844,0.0,0.0,2.000000,1.992157,1.984314
1,1952069,1952069,0.0,0.0,0.000000,NaN,0.992157
2,2976181,2976181,0.0,0.0,2.000000,1.086275,2.000000
3,1684175,1684175,0.0,0.0,1.992157,1.992157,1.952941
4,5528913,5528913,0.0,0.0,0.992157,0.992157,1.000000
...,...,...,...,...,...,...,...
487274,5115269,5115269,0.0,0.0,NaN,0.000000,0.000000
487275,2575244,2575244,0.0,0.0,NaN,NaN,NaN
487276,4704852,4704852,0.0,0.0,0.000000,0.000000,1.000000
487277,1512233,1512233,0.0,0.0,0.000000,0.000000,NaN


In [17]:
#ACE1
ACE1_snps = [
            'chr17:63480412:A:C:rs3730025 ',
            'chr17:63493576:T:C:rs4976',
            'chr17:63493936:T:G:rs4977',
            'chr17:63496854:C:T:rs12709442']

multipleSNPdosages = getVariantDosagesfromSnp(ACE1_snps)
multipleSNPdosages

Variant rs3730025  not found. Skipping.
Variant rs4977 not found. Skipping.


,FID,IID,rs116112765,rs4976,rs12709442
0,3253844,3253844,0.0,0.0,0.0
1,1952069,1952069,0.0,0.0,0.0
2,2976181,2976181,0.0,0.0,0.0
3,1684175,1684175,0.0,0.0,0.0
4,5528913,5528913,0.0,0.0,0.0
...,...,...,...,...,...
487274,5115269,5115269,0.0,0.0,0.0
487275,2575244,2575244,0.0,0.0,0.0
487276,4704852,4704852,0.0,0.0,0.0
487277,1512233,1512233,0.0,0.0,0.0


In [14]:
#BetaBlockers
BB = pd.read_csv('/mnt/project/TTN/dosage_extraction/BB_IVs.txt', delimiter='\t')

BB['variant_id'] = BB[list(BB.columns)].apply(lambda row: ':'.join(row.values.astype(str)), axis=1)
multipleSNPdosages = getVariantDosagesfromSnp(BB['variant_id'].tolist())
multipleSNPdosages

,FID,IID,rs11196549,rs11196597,rs17875473,rs4359161,rs460718
0,3253844,3253844,0.000000,1.0,0.0,0.000000,1.000000
1,1952069,1952069,0.000000,0.0,1.0,0.000000,1.000000
2,2976181,2976181,0.952941,0.0,0.0,0.000000,1.000000
3,1684175,1684175,0.000000,0.0,0.0,1.000000,0.929412
4,5528913,5528913,0.000000,0.0,1.0,0.000000,1.992157
...,...,...,...,...,...,...,...
487274,5115269,5115269,0.000000,0.0,0.0,0.000000,NaN
487275,2575244,2575244,0.015686,1.0,0.0,0.039216,1.000000
487276,4704852,4704852,0.000000,0.0,1.0,0.000000,2.000000
487277,1512233,1512233,0.000000,0.0,0.0,1.000000,1.000000


In [16]:
#Calcum Channel Blockers
CCB = pd.read_csv('/mnt/project/TTN/dosage_extraction/CCB_IVs.txt', delimiter='\t')

CCB['variant_id'] = CCB[list(CCB.columns)].apply(lambda row: ':'.join(row.values.astype(str)), axis=1)

#filtered = CCB[CCB['variant_id'] != 'chr10:18440926:G:C:rs72786098'] #Filter out incorrect entry

multipleSNPdosages = getVariantDosagesfromSnp(CCB['variant_id'].tolist())
multipleSNPdosages

,FID,IID,rs10828399,rs10828542,rs11014170,rs112133583,rs113210396,rs114987861,rs16916914,rs1779209,...,rs2239046,rs2488136,rs3821843,rs4748474,rs61278674,rs7076319,rs714277,rs72786098,rs7340705,rs7923191
0,3253844,3253844,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,2.000000,...,2.000000,0.000000,1.945098,1.984314,0.000000,0.000000,1.000000,0.0,0.015686,0.000000
1,1952069,1952069,0.992157,2.0,0.0,0.000000,0.0,0.0,0.0,1.992157,...,1.000000,2.000000,1.976471,1.000000,0.015686,0.968627,0.000000,0.0,0.992157,1.000000
2,2976181,2976181,2.000000,0.0,0.0,1.000000,0.0,0.0,1.0,2.000000,...,1.000000,2.000000,0.960784,0.000000,0.000000,2.000000,1.000000,0.0,1.984314,0.000000
3,1684175,1684175,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,1.000000,...,1.000000,2.000000,1.058824,0.000000,1.000000,0.000000,1.000000,0.0,1.968627,0.000000
4,5528913,5528913,1.000000,1.0,0.0,0.000000,1.0,0.0,0.0,2.000000,...,2.000000,0.000000,NaN,1.000000,0.000000,1.000000,0.000000,0.0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487274,5115269,5115269,1.000000,1.0,0.0,0.000000,0.0,0.0,0.0,2.000000,...,0.952941,1.000000,NaN,1.047059,0.000000,0.000000,0.937255,0.0,1.000000,0.000000
487275,2575244,2575244,0.992157,NaN,0.0,0.992157,0.0,0.0,0.0,1.968627,...,1.968627,1.952941,NaN,1.000000,0.031373,0.000000,0.968627,0.0,0.015686,0.007843
487276,4704852,4704852,0.992157,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.984314,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.0,2.000000,0.000000
487277,1512233,1512233,1.000000,1.0,0.0,0.000000,0.0,0.0,0.0,1.000000,...,2.000000,2.000000,0.000000,0.000000,1.000000,2.000000,1.000000,0.0,0.000000,1.000000


In [14]:
#IVs_Refined
IV_snps = ['chr10:114029616:C:T:rs7076938',
           'chr5:128358304:C:T:rs78736765',
           'chr4:45954659:C:T:rs7699135',
           'chr10:113947539:G:A:rs11196549',
           'chr10:113961605:A:G:rs460718',
           'chr11:1984564:C:A:rs71455916',
           'chr2:152069747:G:T:rs17326118',
           'chr3:53528867:G:A:rs6445583'
           ]

multipleSNPdosages = getVariantDosagesfromSnp(IV_snps)
multipleSNPdosages

Variant rs71455916 not found. Skipping.


,FID,IID,rs7076938,rs78736765,rs7699135,rs11196549,rs460718,rs17326118,rs6445583
0,3253844,3253844,2.000000,1.000000,1.000000,0.000000,1.000000,2.000000,1.952941
1,1952069,1952069,2.000000,0.000000,0.984314,0.000000,1.000000,1.992157,1.976471
2,2976181,2976181,1.000000,0.000000,2.000000,0.952941,1.000000,1.000000,0.960784
3,1684175,1684175,NaN,0.000000,2.000000,0.000000,0.929412,1.000000,0.984314
4,5528913,5528913,1.992157,0.000000,2.000000,0.000000,1.992157,1.039216,1.945098
...,...,...,...,...,...,...,...,...,...
487274,5115269,5115269,0.984314,0.015686,1.945098,0.000000,NaN,0.984314,NaN
487275,2575244,2575244,1.968627,2.000000,NaN,0.015686,1.000000,1.000000,NaN
487276,4704852,4704852,1.992157,1.000000,1.000000,0.000000,2.000000,2.000000,1.000000
487277,1512233,1512233,1.000000,0.000000,0.960784,0.000000,1.000000,1.000000,1.000000


In [15]:
#IVs_Refined
James_Other_snps = ['chr13:113140972:G:A:rs61753266',
                   'chr1:117025878:G:A:rs12093834', 
                   'chr3:38603929:T:C:rs1805124',
                   'chr8:42187447:G:A:rs2020921',
                   'chr15:73343504:C:T:rs151004999',
                   'chr1:55054735:G:C:rs553741',
                   'chr8:42147724:A:C:rs60393671']

multipleSNPdosages = getVariantDosagesfromSnp(James_Other_snps)
multipleSNPdosages

,FID,IID,rs61753266,rs12093834,rs1805124,rs2020921,rs151004999,rs553741,rs60393671
0,3253844,3253844,0.0,0.0,1.0,0.0,0.0,2.000000,0.0
1,1952069,1952069,0.0,0.0,0.0,0.0,0.0,2.000000,0.0
2,2976181,2976181,0.0,0.0,0.0,0.0,0.0,2.000000,0.0
3,1684175,1684175,0.0,0.0,0.0,0.0,0.0,2.000000,0.0
4,5528913,5528913,0.0,0.0,0.0,0.0,0.0,1.000000,0.0
...,...,...,...,...,...,...,...,...,...
487274,5115269,5115269,0.0,0.0,1.0,0.0,0.0,0.992157,0.0
487275,2575244,2575244,0.0,0.0,1.0,0.0,0.0,1.984314,0.0
487276,4704852,4704852,0.0,0.0,1.0,0.0,0.0,1.000000,0.0
487277,1512233,1512233,0.0,0.0,2.0,0.0,0.0,1.000000,0.0


In [ ]:
#HDAC6_Refined
hdac6 = ['chrX:48822777:G:A:rs61753266',
            'chrX:48823730:G:A:rs41312114']

multipleSNPdosages = getVariantDosagesfromSnp(hdac6)
multipleSNPdosages

In [ ]:
vlcad_acadm = ['chr17:7217786:C:A:rs72839706',
               'chr1:75740000:T:G:rs78392995']
multipleSNPdosages = getVariantDosagesfromSnp(vlcad_acadm)
multipleSNPdosages

In [36]:
FKBP7 = ['chr2:178478419:CTTTT:C:rs574620893']
multipleSNPdosages = getVariantDosagesfromSnp(FKBP7)
multipleSNPdosages

,FID,IID,rs574620893
0,3253844,3253844,0.0
1,1952069,1952069,1.0
2,2976181,2976181,0.0
3,1684175,1684175,0.0
4,5528913,5528913,0.0
...,...,...,...
487274,5115269,5115269,0.0
487275,2575244,2575244,0.0
487276,4704852,4704852,0.0
487277,1512233,1512233,0.0


In [37]:
PTK2 = ['chr8:140663317:C:G:rs13275969']
multipleSNPdosages = getVariantDosagesfromSnp(PTK2)
multipleSNPdosages

,FID,IID,rs13275969
0,3253844,3253844,NaN
1,1952069,1952069,1.000000
2,2976181,2976181,2.000000
3,1684175,1684175,1.000000
4,5528913,5528913,2.000000
...,...,...,...
487274,5115269,5115269,1.976471
487275,2575244,2575244,0.913725
487276,4704852,4704852,1.000000
487277,1512233,1512233,2.000000


In [18]:
TTN = ['chr2:178693639:T:C:rs2042995']
multipleSNPdosages = getVariantDosagesfromSnp(TTN)
multipleSNPdosages

/tmp/ipykernel_98/1515039186.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sample_df = pd.read_csv(sample_path, sep=' ')


,FID,IID,rs2042995
0,4944151,4944151,1.0
1,3046793,3046793,1.0
2,2356393,2356393,0.0
3,2856702,2856702,1.0
4,4994292,4994292,1.0
...,...,...,...
469830,1821983,1821983,1.0
469831,1057517,1057517,1.0
469832,3875359,3875359,0.0
469833,4501687,4501687,0.0


In [21]:
multipleSNPdosages['rs2042995'].value_counts()

0.0    275441
1.0    165867
2.0     28440
Name: rs2042995, dtype: int64

In [22]:
file_path = 'TTN_rs2042995.txt'
multipleSNPdosages.to_csv(file_path, sep='\t', index=False)

In [23]:
%%bash 
dx upload --destination ./TTN/dosage_extraction/ *.txt

ID                          file-Ggy9KY0Jz88Y094q6ZXVyBQ7
Class                       file
Project                     project-GQ9jJq0Jz88vkz66zppF73QP
Folder                      /TTN/dosage_extraction
Name                        TTN_rs2042995.txt
State                       closing
Visibility                  visible
Types                       -
Properties                  -
Tags                        -
Outgoing links              -
Created                     Thu Mar 21 20:23:40 2024
Created by                  lzhang
 via the job                job-Ggy96g0Jz88k390QPqZppz8F
Last modified               Thu Mar 21 20:23:41 2024
Media type                  
archivalState               "live"
cloudAccount                "cloudaccount-dnanexus"
